# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Джемадинов Эскендер Рустемович`

Группа: `MADE-ML-32`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [1]:
!pip install reverse_geocoder

# Двузначные коды стран с полным назыанием
!wget https://datahub.io/core/country-list/r/data.csv

    
!pip install folium

You should consider upgrading via the '/home/eskender/anaconda3/bin/python -m pip install --upgrade pip' command.
--2020-11-05 01:20:00--  https://datahub.io/core/country-list/r/data.csv
Resolving datahub.io (datahub.io)... 104.18.49.253, 104.18.48.253, 172.67.157.38, ...
Connecting to datahub.io (datahub.io)|104.18.49.253|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv [following]
--2020-11-05 01:20:01--  https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv
Resolving pkgstore.datahub.io (pkgstore.datahub.io)... 104.18.49.253, 104.18.48.253, 172.67.157.38, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|104.18.49.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4120 (4,0K) [text/plain]
Saving to: ‘data.csv.2’

data.csv.2          100%[===================>]   

In [2]:
from random import uniform
import geocoder

import shapefile
import reverse_geocoder as rg 
import pandas as pd

import geopandas as gpd
from shapely import wkt

from shapely.geometry.point import Point

In [3]:
def newpoint():
    return uniform(-180,180), uniform(-90, 90)

def reverseGeocode(coordinates): 
    result = rg.search(coordinates) 
      
    return result


In [4]:
countries = pd.read_csv("data.csv")

countries = dict(zip(countries.Code, countries.Name))


In [5]:
countries['AF']

'Afghanistan'

In [7]:
# Наивное решение, простая генерация. 

geo = ""

while geo != 'Afghanistan':
    
    coordinates =newpoint()
    
    location = reverseGeocode(coordinates)

    if location[0]['cc'] in countries.keys():
        geo = countries[location[0]['cc']]
        print("Random point! We're in {0}".format(geo))

Random point! We're in Russian Federation
Random point! We're in United States
Random point! We're in South Africa
Random point! We're in Russian Federation
Random point! We're in Svalbard and Jan Mayen
Random point! We're in Saint Pierre and Miquelon
Random point! We're in Svalbard and Jan Mayen
Random point! We're in South Africa
Random point! We're in French Southern Territories
Random point! We're in Brazil
Random point! We're in French Southern Territories
Random point! We're in Chad
Random point! We're in French Southern Territories
Random point! We're in Chile
Random point! We're in Svalbard and Jan Mayen
Random point! We're in Russian Federation
Random point! We're in French Southern Territories
Random point! We're in Madagascar
Random point! We're in Portugal
Random point! We're in Mauritania
Random point! We're in Libya
Random point! We're in Greenland
Random point! We're in Argentina
Random point! We're in Greenland
Random point! We're in Svalbard and Jan Mayen
Random point!

Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in French Southern Territories
Random point! We're in Cape Verde
Random point! We're in Canada
Random point! We're in Russian Federation
Random point! We're in Argentina
Random point! We're in Argentina
Random point! We're in French Southern Territories
Random point! We're in Germany
Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in Svalbard and Jan Mayen
Random point! We're in Greenland
Random point! We're in Mauritius
Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in Brazil
Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in Antigua and Barbuda
Random point! We're in Somalia
Random point! We're in Greenland
Random point! We're in South Georgia a

Random point! We're in Greenland
Random point! We're in French Southern Territories
Random point! We're in Argentina
Random point! We're in Russian Federation
Random point! We're in Greece
Random point! We're in Svalbard and Jan Mayen
Random point! We're in Brazil
Random point! We're in Argentina
Random point! We're in French Southern Territories
Random point! We're in South Africa
Random point! We're in Falkland Islands (Malvinas)
Random point! We're in South Georgia and the South Sandwich Islands
Random point! We're in South Africa
Random point! We're in Libya
Random point! We're in Greenland
Random point! We're in Iceland
Random point! We're in Chile
Random point! We're in French Southern Territories
Random point! We're in French Southern Territories
Random point! We're in Portugal
Random point! We're in Svalbard and Jan Mayen
Random point! We're in Argentina
Random point! We're in Canada
Random point! We're in Russian Federation
Random point! We're in French Southern Territories
Ra

In [8]:
countries_pol = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))\
                .rename(columns={"name":"country_name"})
poly = wkt.loads(str(countries_pol[countries_pol.country_name=='Afghanistan']['geometry'].values[0]))


In [22]:
# Продвинутое решение. Генерируем точку (x,y). Далее генерируем точку в окрестности(okr_x, okr_y) 
# с координатами (x_, y_), если она ближе идём к ней, нет, в противоположном направлении
# (не обязательно в ближайшем). При этом уменьшаем шаг диапазона на параметр alpha (чтобы не ходить туда сюда).  

def newpoint_corr(x, y, okr_x, okr_y):
    return uniform(max(x-okr_x,-180), min(x+okr_x, 180)),  uniform(max(y- okr_y, -90), min(y+okr_y, 90))

geo = ""

latlon =[]

okr_x = 10
okr_y = 10
alpha = 0.99

x, y = newpoint()
x_ = 0 
y_ = 0

while geo != 'Afghanistan':
    
    x_, y_ = newpoint_corr(x, y, okr_x, okr_y)        
    pt = wkt.loads("POINT({} {})".format(x_, y_))    
    pt_pred = wkt.loads("POINT({} {})".format(x, y))    
    
    if poly.distance(pt) < poly.distance(pt_pred):

        x, y = x_, y_
          
        location = reverseGeocode((y, x))

    else: 

        x = x - (x-x_)*-1
        y = y - (y-y_)*-1            

        location = reverseGeocode((y, x))
        
    latlon.append((x, y))
        
    if location[0]['cc'] in countries.keys():
        geo = countries[location[0]['cc']]

        okr_x = okr_x*alpha
        okr_y = okr_y*alpha
        print("loss: {0}. We're in {1} currenly".format(poly.distance(pt), countries[location[0]['cc']]))


loss: 42.90550503801659. We're in Maldives currenly
loss: 42.73849962895207. We're in Mauritius currenly
loss: 46.17906065821789. We're in Maldives currenly
loss: 42.401021664737165. We're in Maldives currenly
loss: 41.270930807728185. We're in Sri Lanka currenly
loss: 29.9574894845912. We're in Sri Lanka currenly
loss: 31.36500448226785. We're in Sri Lanka currenly
loss: 29.79442319688076. We're in Maldives currenly
loss: 22.52837126646198. We're in Maldives currenly
loss: 22.11112790711275. We're in Maldives currenly
loss: 28.395637664536682. We're in Yemen currenly
loss: 21.853232767270352. We're in Oman currenly
loss: 8.095224995041423. We're in Oman currenly
loss: 6.352490381348989. We're in Pakistan currenly
loss: 4.1150775391303664. We're in Pakistan currenly
loss: 3.8911459419276744. We're in Iran, Islamic Republic of currenly
loss: 1.992646036549426. We're in Uzbekistan currenly
loss: 0.0. We're in Afghanistan currenly


Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

In [23]:
import folium

In [24]:
mapit = folium.Map( location=[ latlon[0][1], latlon[0][0] ] )

for index, coord in enumerate(latlon):
    if index < len(latlon) -1:  
        folium.Marker( location=[ coord[1], coord[0] ], fill_color='#43d9de', radius=7 ).add_to( mapit )
    else:
        folium.Marker( location=[ coord[1], coord[0]] ,popup='Some Other Location',
                                 icon=folium.Icon(color='red', icon='info-sign')).add_to(mapit)

In [25]:
mapit

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here